# Assignment 3
- Zeynep Türkmen - 29541

### Importing the libraries

In [2]:
from google.colab import drive
import os
import cv2
import numpy as np
from scipy.spatial import distance
import pandas as pd
from itertools import combinations

!pip install mahotas
import mahotas

### Reading the images into arrays

In [3]:
#mounting the google drive to retrieve photos
drive.mount('/content/drive')

#train folder directory
train_folder = '/content/drive/My Drive/419/hw3/train'
train_images_list = os.listdir(train_folder)

#test folder directory
test_folder = '/content/drive/My Drive/419/hw3/test'
test_images_list = os.listdir(test_folder)

Mounted at /content/drive


In [4]:
#reading the images into their corresponding arrays by also adding corresponding labels
train_list = []

for image_name in train_images_list:
  img_path = os.path.join(train_folder, image_name)
  img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
  train_list.append({'image': img, 'label': image_name.split('-')[0]})#seperating the class name from the file name and saving it

In [5]:
#reading the test images
test_list = []

for image_name in test_images_list:
  img_path = os.path.join(test_folder, image_name)
  img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
  test_list.append({'image': img, 'label': image_name.split('-')[0]})#seperating the class name from the file name and saving it

### Implementing the shape descriptors

#### Area

In [6]:
def get_area(img):
  area = cv2.countNonZero(img)#count the nonzero pixels in the binary image and it simply gives the area
  return area

#### Perimeter

In [7]:
def get_perimeter(img, contours):
  if not contours:
    return 0 #no borders no nothing just blackness
  else:
    max_border = max(contours, key=cv2.contourArea) #get the max area contour aka the largest border, which might not necessarily hold for all types of it being the "perimeter" but still works for the given image set :)
    perimeter = cv2.arcLength(max_border, True) #assuming its a closed curve by nature calculating the length
    return perimeter

#### Convexity

In [8]:
def get_convexity(img, contours, area):
  if not contours:
    return 0 #no borders no nothing just blackness
  else:
    border = max(contours, key=cv2.contourArea) #theres a single object in all images so theres only one contour as we avoided internal ones i think
    #get the convex hull based on the border of the image
    convex_hull = cv2.convexHull(border)
    #calculate the convex hull versions area
    area_convex = cv2.contourArea(convex_hull)
    #ratio of it gives "how convex" the item is
    convexity = area / area_convex

    return convexity

#### Circularity

In [9]:
def get_circularity(img, area, perimeter):
  #considering its area and perimeter simply get a ratio as to how much it resembles a circle
  circularity = (perimeter ** 2)/area
  return circularity

#### Rectangularity

In [10]:
def get_rectangularity(img, contours, area):
  if not contours:
    return 0  #no borders no nothing just blackness
  else:
    border = max(contours, key=cv2.contourArea)
    #retrieving the rectangle based on the contour
    _, _, width, height = cv2.boundingRect(border)

    #get the areas of both the original img and the rectangle surrounding it
    area_of_rectangle = width * height

    #calculate how rectangle it is based on the area ratio
    if area_of_rectangle != 0:
      rectangularity = area / area_of_rectangle
      return rectangularity
    else:
      return 0

#### Eccentricity

In [11]:
def get_eccentricity(img, contours): #oohhh that weird formula
  if not contours:
    return 0
  else:
    border = max(contours, key=cv2.contourArea) #theres a single item in the image
    #get the covariance matrix belonging to the borders
    covariances = np.cov(border[:, 0, :], rowvar=False)
    #calculate the eigenvalues corresponding to that
    eigenval, _ = np.linalg.eigh(covariances)

    #sort the values to get the big one for major_axis
    eigenval = np.sort(eigenval)[::-1]

    #get the ratio for eccentricity
    #its a 2D image so there are only 2 eigenvalues anyway
    minor_axis = np.sqrt(eigenval[1])
    major_axis = np.sqrt(eigenval[0])

    eccentricity = np.sqrt(1 - (minor_axis / major_axis)**2)
    return eccentricity

#### Fourier Descriptor

In [12]:
def get_fourier_descriptor(img, contours, coefficient_count): #to find different versions with various coefficient counts
  if not contours:
    return 0
  else:
    border = max(contours, key=cv2.contourArea)

    #converting them to complex values to be suitable for the fourier transformation
    complex_values = []
    for point in border:
      complex_values.append(complex(point[0][0], point[0][1]))

    #apply the transform
    f = np.fft.fft(complex_values)

    #keep a certain number of coefficient
    #here as i remember the lower order ones represent the shape so I start chopping the ends accordingly and keep the beginning
    fourier_descriptor = f[:coefficient_count]

    #if theres not enough descriptors based on the given amount padd it to make it equal for comparison later on
    length = len(fourier_descriptor)
    if length < coefficient_count:
      fourier_descriptor = np.pad(fourier_descriptor, (0, coefficient_count - length))

    return fourier_descriptor

#### Shape Histograms

In [13]:
def get_shape_histogram(img, contours, bin_count, area): #take the area to normalize the histograms to make it scale invariant
  if not contours:
    return 0
  else:
    border = max(contours, key=cv2.contourArea)

    #find the center of mass coordinates of the image
    moments = cv2.moments(border)
    x = int(moments['m10'] / moments['m00'])
    y = int(moments['m01'] / moments['m00'])

    #find the farthest point from the center to draw the biggest concentric circle
    max_circle = 0
    for point in border:
      radius = np.linalg.norm(np.array(point[0]) - np.array([x, y]))
      max_circle = max(max_circle, radius)


    bins = np.linspace(0, max_circle, bin_count + 1) #create equally spaced bins between 0 and the maximum point based on the given bin count

    distances = []
    #calculate the distance to the center point for all border pixels
    for point in border:
      current_distance = np.linalg.norm(np.array(point[0]) - np.array([x, y]))
      distances.append(current_distance)

    #create the dedicated histogram
    histogram, _ = np.histogram(distances, bins)
    return histogram/area

#### Moment invariants

In [14]:
def get_moment_invariants(img):
  moments = cv2.moments(img)
  #moments describe the shape

  hu = cv2.HuMoments(moments).flatten()
  zernike = mahotas.features.zernike_moments(img, 8)

  #extracting the central moments
  central = np.array([
    moments['mu20'], moments['mu11'], moments['mu02'],
    moments['mu30'], moments['mu21'], moments['mu12'], moments['mu03']
  ])

  #return a dict containing different types of moments that may be suitable for shape recognition
  titles = ['hu_moments', 'zernike_moments', 'central_moments']
  values = [hu, zernike, np.array(list(central))]

  all_moments = dict(zip(titles, values))
  return all_moments

### Creating global descriptors

In [15]:
def get_global_shape_descriptor_basic(img): #here I will create an array of all the descriptors for all the images and modify which feature to encorporate later on..
  #appearently contours give a curve joining continuous points along a border with same intensity, aka gives the boundary
  contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) #only interested in the main image so RETR_EXTERNAL parameter helps avoid holes within the object

  area = get_area(img)
  perimeter = get_perimeter(img, contours)
  convexity = get_convexity(img, contours, area)
  circularity = get_circularity(img, area, perimeter)
  rectangularity = get_rectangularity(img, contours, area)
  eccentricity = get_eccentricity(img, contours)

  area = np.array([area])
  perimeter = np.array([perimeter])
  convexity = np.array([convexity])
  circularity = np.array([circularity])
  rectangularity = np.array([rectangularity])
  eccentricity = np.array([eccentricity])

  #im gonna store them with their titles and save it to not wait 100 hours for the code to run each time
  titles = ['area', 'perimeter', 'convexity', 'circularity', 'rectangularity', 'eccentricity']
  values = [area, perimeter, convexity, circularity, rectangularity, eccentricity]

  global_descriptor_basic = {title: value for title, value in zip(titles, values)}
  return global_descriptor_basic

In [16]:
def get_global_shape_descriptor_fourier(img): #here I will create an array of all the descriptors for all the images and modify which feature to encorporate later on..
  #appearently contours give a curve joining continuous points along a border with same intensity, aka gives the boundary
  contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) #only interested in the main image so RETR_EXTERNAL parameter helps avoid holes within the object

  fourier_5 = get_fourier_descriptor(img, contours, 5)
  fourier_10 = get_fourier_descriptor(img, contours, 10)
  fourier_15 = get_fourier_descriptor(img, contours, 15)
  fourier_20 = get_fourier_descriptor(img, contours, 20)
  fourier_25 = get_fourier_descriptor(img, contours, 25)

  #im gonna store them with their titles and save it to not wait 100 hours for the code to run each time
  titles = ['fourier_5', 'fourier_10', 'fourier_15', 'fourier_20', 'fourier_25']
  values = [fourier_5, fourier_10, fourier_15, fourier_20, fourier_25]

  global_descriptor_fourier = {title: value for title, value in zip(titles, values)}
  return global_descriptor_fourier

In [17]:
def get_global_shape_descriptor_histogram(img): #here I will create an array of all the descriptors for all the images and modify which feature to encorporate later on..
  #appearently contours give a curve joining continuous points along a border with same intensity, aka gives the boundary
  contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) #only interested in the main image so RETR_EXTERNAL parameter helps avoid holes within the object
  area = get_area(img)

  hist_5 = get_shape_histogram(img, contours, 5, area)
  hist_10 = get_shape_histogram(img, contours, 10, area)
  hist_15 = get_shape_histogram(img, contours, 15, area)
  hist_20 = get_shape_histogram(img, contours, 20, area)
  hist_25 = get_shape_histogram(img, contours, 25, area)

  #im gonna store them with their titles and save it to not wait 100 hours for the code to run each time
  titles = ['hist_5', 'hist_10', 'hist_15', 'hist_20', 'hist_25']
  values = [hist_5, hist_10, hist_15, hist_20, hist_25]

  global_descriptor_histogram = {title: value for title, value in zip(titles, values)}
  return global_descriptor_histogram

### Storing all the feature data

In [18]:
train_features = []

for train_img in train_list:
  basic_descriptor = get_global_shape_descriptor_basic(train_img['image'])
  fourier_descriptor = get_global_shape_descriptor_fourier(train_img['image'])
  histogram_descriptor = get_global_shape_descriptor_histogram(train_img['image'])
  moment_descriptor = get_moment_invariants(train_img['image'])

  #merge all 3 dictionaries
  descriptor = {**basic_descriptor, **fourier_descriptor, **histogram_descriptor, **moment_descriptor}
  train_features.append({'descriptor': descriptor, 'label': train_img['label']})

#now train features consists of a list of dictionaries that have all the information for all the train images so lets store that
np.save('train_features.npy', train_features)

In [ ]:
#to help me pick up from where I left off :d
train_features = np.load('train_features.npy', allow_pickle=True)

In [ ]:
train_features

Output hidden; open in https://colab.research.google.com to view.

In [19]:
#same exact thing for test images
test_features = []

for test_img in test_list:
  basic_descriptor = get_global_shape_descriptor_basic(test_img['image'])
  fourier_descriptor = get_global_shape_descriptor_fourier(test_img['image'])
  histogram_descriptor = get_global_shape_descriptor_histogram(test_img['image'])
  moment_descriptor = get_moment_invariants(test_img['image'])

  #merge all 3 dictionaries
  descriptor = {**basic_descriptor, **fourier_descriptor, **histogram_descriptor, **moment_descriptor}
  test_features.append({'descriptor': descriptor, 'label': test_img['label']})

#now test features consists of a list of dictionaries that have all the information for all the test images so lets store that
np.save('test_features.npy', test_features)

In [ ]:
test_features

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#to help me pick up from where I left off :d, didnt need it at all but meh
test_features = np.load('test_features.npy', allow_pickle=True)

### Classifying the objects: distance calculator & the classifier

In [20]:
#just to blend sublists
def flatten_list(list_nested):
  flat = []
  for item in list_nested:
    if isinstance(item, np.ndarray):
      for sub in item:
        flat.append(sub)
    else:
      flat.append(item)
  return flat

In [21]:
def distance_calculator(val1, val2, distance_metric, covariance_matrix):

  the_distance = None
  if distance_metric == 'euclidean':
    the_distance = np.linalg.norm(val1 - val2)
  elif distance_metric == 'manhattan':
    the_distance = distance.cityblock(val1, val2)
  elif distance_metric == 'chisquared':
    e = 1e-10 #avoiding division by 0
    the_distance = np.sum((val1 - val2)**2 / (val1 + val2 + e))
  elif distance_metric == 'mahalanobis':
    e = 1e-10  # Avoiding singular matrices
    cov_avoid_singular = covariance_matrix + e * np.identity(covariance_matrix.shape[0])
    try:
        precision_matrix = np.linalg.inv(cov_avoid_singular)
        the_distance = distance.mahalanobis(val1, val2, precision_matrix)
    except np.linalg.LinAlgError:
        try:
            precision_matrix = np.linalg.pinv(cov_avoid_singular)
            the_distance = distance.mahalanobis(val1, val2, precision_matrix)
        except np.linalg.LinAlgError:
            print("still singular yikes")
  return the_distance

In [22]:
def classify_objects(training_set, testing_set, distance_metric, feature_list):
    correct = 0

    testing_features = []
    training_features = []

    for item_t in testing_set:
      testing_features.append([item_t['descriptor'][feature] for feature in feature_list])

    for item_t in training_set:
      training_features.append([item_t['descriptor'][feature] for feature in feature_list])

    unpacked_test = np.array([flatten_list(item) for item in testing_features])
    unpacked_train = np.array([flatten_list(item) for item in training_features])


    covariance_matrix = np.cov(unpacked_train, rowvar=False)

    for test_idx, test_item in enumerate(unpacked_test):
      #find the closest one to the test one inside the items we trained with, USING the inputted distance metric and based on the GIVEN feature list

      closest = None
      closest_idx = None
      current_min = float('inf')  # Initialize min_sum to positive infinity

      for train_idx, train_item in enumerate(unpacked_train):
        current_sum = distance_calculator(train_item, test_item, distance_metric, covariance_matrix)

        if current_sum < current_min:
          current_min = current_sum
          closest = train_item
          closest_idx = train_idx

      #prediction is the label of the closest one
      prediction = training_set[closest_idx]['label']

      if prediction == testing_set[test_idx]['label']: #if it guessed correctly
        correct += 1

    # calculate the ratio of correctly classified test samples
    accuracy = correct / len(testing_set)

    return accuracy

### Experimenting with different features

In [23]:
#list of all features
features = ['area', 'perimeter', 'convexity', 'circularity', 'rectangularity', 'eccentricity', 'fourier_5', 'fourier_10', 'fourier_15', 'fourier_20', 'fourier_25', 'hist_5', 'hist_10', 'hist_15', 'hist_20', 'hist_25', 'hu_moments', 'zernike_moments', 'central_moments']
#I ran this code many many times and exported countless csv files AND
#eccentricity & some other stuff didnt seem to add much to accuracy so i just removed them for further experiments
features = ['area', 'perimeter', 'convexity', 'circularity', 'rectangularity', 'fourier_10', 'hist_10', 'hu_moments', 'central_moments']
subset_list = []

#experimenting with various subsets and
for i in range(4, len(features) + 1):
    for subset in combinations(features, i):
      hist = sum(item.startswith('hist') for item in subset)
      fourier = sum(item.startswith('fourier') for item in subset)
      moments = sum(item.endswith('moments') for item in subset)

      if hist <= 1 and fourier <= 1 and moments <= 1:
        subset_list.append(subset)

In [24]:
#list of all distances
distance_metrics = ['euclidean', 'manhattan', 'mahalanobis', 'chisquared']

#a panda df to store results
results = pd.DataFrame(columns=['Subset', 'Distance Metric', 'Accuracy'])

#all possible subsets with at least 3 features
for subset in subset_list:
  subset_name = "-".join(subset)#adding the subset name to panda
  #try all distance metrics for this subset
  for distance_metric in distance_metrics:
    accuracy = classify_objects(train_features, test_features, distance_metric, list(subset))
    #add it to the dataframe
    results = pd.concat([results, pd.DataFrame({'Subset': [subset_name], 'Distance Metric': [distance_metric], 'Accuracy': [accuracy]})], ignore_index=True)

# Display the results dataframe
print(results)

                                                Subset Distance Metric  \
0                 area-perimeter-convexity-circularity      chisquared   
1              area-perimeter-convexity-rectangularity      chisquared   
2                  area-perimeter-convexity-fourier_10      chisquared   
3                     area-perimeter-convexity-hist_10      chisquared   
4                  area-perimeter-convexity-hu_moments      chisquared   
..                                                 ...             ...   
257  area-convexity-circularity-rectangularity-four...      chisquared   
258  perimeter-convexity-circularity-rectangularity...      chisquared   
259  perimeter-convexity-circularity-rectangularity...      chisquared   
260  area-perimeter-convexity-circularity-rectangul...      chisquared   
261  area-perimeter-convexity-circularity-rectangul...      chisquared   

     Accuracy  
0    0.327143  
1    0.322857  
2    0.031429  
3    0.322857  
4    0.321429  
..        ...  

### Just some random results

In [ ]:
results

,Subset,Distance Metric,Accuracy
0,area-perimeter-convexity-circularity,euclidean,0.280000
1,area-perimeter-convexity-circularity,manhattan,0.298571
2,area-perimeter-convexity-circularity,mahalanobis,0.478571
3,area-perimeter-convexity-circularity,chisquared,0.321429
4,area-perimeter-convexity-rectangularity,euclidean,0.280000
...,...,...,...
1043,area-perimeter-convexity-circularity-rectangul...,chisquared,0.004286
1044,area-perimeter-convexity-circularity-rectangul...,euclidean,0.484286
1045,area-perimeter-convexity-circularity-rectangul...,manhattan,0.520000
1046,area-perimeter-convexity-circularity-rectangul...,mahalanobis,0.108571


In [ ]:
df_sorted = results.sort_values(by='Accuracy', ascending=False)
df_sorted #this is the result of many many experiments and combinations

,Subset,Distance Metric,Accuracy
902,area-convexity-circularity-rectangularity-hist...,mahalanobis,0.710000
946,perimeter-convexity-circularity-rectangularity...,mahalanobis,0.710000
994,area-perimeter-convexity-circularity-rectangul...,mahalanobis,0.710000
790,area-perimeter-convexity-circularity-rectangul...,mahalanobis,0.710000
998,area-perimeter-convexity-circularity-rectangul...,mahalanobis,0.708571
...,...,...,...
359,convexity-rectangularity-fourier_10-hist_10,chisquared,0.004286
787,area-perimeter-convexity-circularity-rectangul...,chisquared,0.004286
343,convexity-circularity-fourier_10-hu_moments,chisquared,0.004286
339,convexity-circularity-fourier_10-hist_10,chisquared,0.004286


from matplotlib import pyplot as plt
df_sorted['Accuracy'].plot(kind='hist', bins=20, title='Accuracy')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
df_sorted.groupby('Distance Metric').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df_sorted['Accuracy'].plot(kind='line', figsize=(8, 4), title='Accuracy')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df_sorted['Distance Metric'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df_sorted, x='Accuracy', y='Distance Metric', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [27]:
df_sorted.to_csv('results.csv', index=False)

In [26]:
df_sorted = results.sort_values(by='Accuracy', ascending=False)
df_sorted #this is the result of many many experiments and combinations

,Subset,Distance Metric,Accuracy
92,convexity-rectangularity-hist_10-hu_moments,chisquared,0.668571
188,convexity-circularity-rectangularity-hist_10-h...,chisquared,0.387143
81,convexity-circularity-rectangularity-hist_10,chisquared,0.385714
99,circularity-rectangularity-hist_10-hu_moments,chisquared,0.365714
87,convexity-circularity-hist_10-hu_moments,chisquared,0.355714
...,...,...,...
130,area-perimeter-rectangularity-fourier_10-centr...,chisquared,0.017143
127,area-perimeter-circularity-hist_10-central_mom...,chisquared,0.017143
125,area-perimeter-circularity-fourier_10-central_...,chisquared,0.017143
122,area-perimeter-circularity-rectangularity-cent...,chisquared,0.017143


### Final results

In [28]:
wrong_chi_df = pd.read_csv('final_results.csv')
df_filtered = wrong_chi_df[wrong_chi_df['Distance Metric'] != 'chisquared']

true_chi_df = pd.read_csv('chi.csv')

fixed_df = pd.concat([df_filtered, true_chi_df], ignore_index=True)

sorted_df = fixed_df.sort_values(by='Accuracy', ascending=False)
sorted_df.to_csv('actual_final_results.csv', index=False)

In [29]:
sorted_df

,Subset,Distance Metric,Accuracy
0,area-convexity-circularity-rectangularity-hist...,mahalanobis,0.710000
3,area-perimeter-convexity-circularity-rectangul...,mahalanobis,0.710000
1,perimeter-convexity-circularity-rectangularity...,mahalanobis,0.710000
2,area-perimeter-convexity-circularity-rectangul...,mahalanobis,0.710000
4,area-perimeter-convexity-circularity-rectangul...,mahalanobis,0.708571
...,...,...,...
978,area-perimeter-convexity-rectangularity-hist_1...,chisquared,0.017143
977,area-perimeter-circularity-rectangularity-four...,chisquared,0.017143
976,area-perimeter-convexity-fourier_10-hist_10-ce...,chisquared,0.017143
975,area-perimeter-convexity-rectangularity-fourie...,chisquared,0.017143
